## Install OpenAI Library

In [1]:
pip install openai


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tiktoken


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import Library

In [4]:
import openai
import json
import tiktoken
from dotenv import dotenv_values
config = dotenv_values(".env")

## Add API key

In [5]:
openai.api_key  = config["OPENAI_API_KEY"]

In [6]:
## Declare Functions

In [7]:
# enc = tiktoken.get_encoding("cl100k_base")
enc = tiktoken.encoding_for_model("gpt-4")

In [8]:
def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [9]:
price_per_token = 0.002 / 1000

## Insert Your Input

In [10]:
book_text = """
It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.

However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”

Mr. Bennet replied that he had not.

“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”

Mr. Bennet made no answer.

“Do you not want to know who has taken it?” cried his wife impatiently.

“YOU want to tell me, and I have no objection to hearing it.”

This was invitation enough.

“Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr. Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.”

“What is his name?”

“Bingley.”

“Is he married or single?”

“Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!”

“How so? How can it affect them?”

“My dear Mr. Bennet,” replied his wife, “how can you be so tiresome! You must know that I am thinking of his marrying one of them.”

“Is that his design in settling here?”

“Design! Nonsense, how can you talk so! But it is very likely that he MAY fall in love with one of them, and therefore you must visit him as soon as he comes.”

“I see no occasion for that. You and the girls may go, or you may send them by themselves, which perhaps will be still better, for as you are as handsome as any of them, Mr. Bingley may like you the best of the party.”

“My dear, you flatter me. I certainly HAVE had my share of beauty, but I do not pretend to be anything extraordinary now. When a woman has five grown-up daughters, she ought to give over thinking of her own beauty.”

“In such cases, a woman has not often much beauty to think of.”

“But, my dear, you must indeed go and see Mr. Bingley when he comes into the neighbourhood.”

“It is more than I engage for, I assure you.”

“But consider your daughters. Only think what an establishment it would be for one of them. Sir William and Lady Lucas are determined to go, merely on that account, for in general, you know, they visit no newcomers. Indeed you must go, for it will be impossible for US to visit him if you do not.”

“You are over-scrupulous, surely. I dare say Mr. Bingley will be very glad to see you; and I will send a few lines by you to assure him of my hearty consent to his marrying whichever he chooses of the girls; though I must throw in a good word for my little Lizzy.”

“I desire you will do no such thing. Lizzy is not a bit better than the others; and I am sure she is not half so handsome as Jane, nor half so good-humoured as Lydia. But you are always giving HER the preference.”

“They have none of them much to recommend them,” replied he; “they are all silly and ignorant like other girls; but Lizzy has something more of quickness than her sisters.”

“Mr. Bennet, how CAN you abuse your own children in such a way? You take delight in vexing me. You have no compassion for my poor nerves.”

“You mistake me, my dear. I have a high respect for your nerves. They are my old friends. I have heard you mention them with consideration these last twenty years at least.”

Mr. Bennet was so odd a mixture of quick parts, sarcastic humour, reserve, and caprice, that the experience of three-and-twenty years had been insufficient to make his wife understand his character. HER mind was less difficult to develop. She was a woman of mean understanding, little information, and uncertain temper. When she was discontented, she fancied herself nervous. The business of her life was to get her daughters married; its solace was visiting and news.
"""

## Prompt

In [11]:
def run_prompt():
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": "give my 3 joke stories"
            }
        ],
        max_tokens=256,
        temperature=0.7,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    print(completion.choices[0]["message"]["content"])
    print("-----")
    print(completion)

In [12]:
## Execute Functions

In [13]:
len(enc.encode(book_text))

967

In [14]:
tokens = num_tokens_from_string(book_text, "gpt-3.5-turbo")
print(tokens)
print(f"${tokens * price_per_token}")

967
$0.001934


In [15]:
run_prompt()

1. The Talking Dog
A man walks into a bar with his dog and says to the bartender, "I bet you $100 my dog can talk." The bartender, intrigued, agrees to the bet. The man turns to his dog and asks, "What's on top of a house?" The dog barks, "Roof! Roof!" The bartender, unimpressed, tells the man to leave. As they're walking out, the dog turns to the man and says, "Do you think I should have said 'shingles'?"

2. The Chicken and the Egg
A chicken and an egg are lying in bed together. The chicken lights up a cigarette and says, "Well, I guess that answers that question."

3. The Blonde and the Traffic Jam
A blonde is stuck in a traffic jam and decides to call her husband. "Honey," she says, "I'm stuck in a traffic jam and I don't know what to do." Her husband replies, "Just stay calm and stay in your car. The news says there's a crazy driver going the wrong way on the highway." The blonde responds, "Oh my god, there's not just one, there's hundreds!"
-----
{
  "choices": [
    {
      "fin